In [0]:
cd /content/drive/My Drive/data-collection-from-transportation-platforms-morocco

/content/drive/My Drive/data-collection-from-transportation-platforms-morocco


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import optimizers
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input,Dense, concatenate,Dropout,BatchNormalization,Activation,Flatten,Add, Conv2D, Dot, dot
from keras.layers import RepeatVector, merge, Subtract, Lambda, Multiply, Embedding, Concatenate, Reshape, DepthwiseConv2D
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.engine.topology import Layer
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from tensorflow.keras.utils import plot_model

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
LABEL_COLUMN = "rating"

CATEGORICAL_COLUMNS = [
    'from', 'to' ,'session','gender', 'profission','dayofweek'
]

CONTINUOUS_COLUMNS = [
    'distance','duration', 'price','age'
]

In [0]:
def preprocessing():
    data = pd.read_csv('dataset.csv',delimiter =';')
    print(data.shape)
    # Convert the position of each starting city to a number 
    a = data.groupby(['from_lat','from_lan']).size()
    from_dict = {}
    for i,row in enumerate(a.iteritems()):
        from_dict[row[0]] = i
    f = data[['from_lat','from_lan']]
    from_col = []
    for a,b in f.iterrows():
        from_col.append(from_dict[(b['from_lat'],b['from_lan'])])
    data['from'] = from_col
    data = data.drop(['from_lat','from_lan'],axis = 1)
    # Convert the position of each arrival city to a number 
    a = data.groupby(['to_lat','to_lan']).size()
    from_dict = {}
    for i,row in enumerate(a.iteritems()):
        from_dict[row[0]] = i
    f = data[['to_lat','to_lan']]
    from_col = []
    for a,b in f.iterrows():
        from_col.append(from_dict[(b['to_lat'],b['to_lan'])])
    data['to'] = from_col
    data = data.drop(['to_lat','to_lan'],axis = 1)
    
    for c in CATEGORICAL_COLUMNS:
        le = LabelEncoder()
        data[c] = le.fit_transform(data[c])
    # train,validation,test split
    train=data.sample(frac=0.8,random_state=200)
    val = data.drop(train.index).sample(frac = 0.5,random_state=200)
    test=data.drop(train.index).drop(val.index)
    
    y_train = train['rating'].values
    y_val = val['rating'].values
    y_test = test['rating'].values
    x_train = train.drop(['rating'],axis=1)
    x_val = val.drop(['rating'],axis=1)
    x_test = test.drop(['rating'],axis=1)
    x_train_categ = np.array(x_train[CATEGORICAL_COLUMNS]) 
    x_val_categ = np.array(x_val[CATEGORICAL_COLUMNS])
    x_test_categ = np.array(x_test[CATEGORICAL_COLUMNS])
    x_train_conti = np.array(x_train[CONTINUOUS_COLUMNS], dtype='float64') 
    x_val_conti = np.array(x_val[CONTINUOUS_COLUMNS], dtype='float64')
    x_test_conti = np.array(x_test[CONTINUOUS_COLUMNS], dtype='float64')

    scaler = StandardScaler()
    x_train_conti = scaler.fit_transform(x_train_conti)
    x_val_conti = scaler.transform(x_val_conti)    
    x_test_conti = scaler.transform(x_test_conti)
    return [x_train, y_train, x_val, y_val, x_test, y_test, x_train_categ, x_val_categ, x_test_categ, x_train_conti,x_val_conti, x_test_conti, data]

In [0]:
class xDeepMF:
  def __init__(self,hidden_layer, conv_layer, embed_dim):
    self.hidden_layer= hidden_layer # List of sizes of the hidden layers
    self.conv_layer = conv_layer    # List of sizes of the conv layers
    self.embed_dim = embed_dim      # size of embedding vectors
    self.model = None 
    x_train, y_train, x_val, y_val, x_test, y_test, x_train_categ,x_val_categ, x_test_categ, x_train_conti,x_val_conti, x_test_conti, data = preprocessing()
    self.x_train = x_train
    self.y_train = y_train / 5
    self.x_val = x_val
    self.y_val = y_val / 5
    self.x_test = x_test
    self.y_test = y_test / 5
    self.x_train_categ = x_train_categ 
    self.x_val_categ = x_val_categ 
    self.x_test_categ = x_test_categ 
    self.x_train_conti = x_train_conti 
    self.x_test_conti = x_test_conti 
    self.x_val_conti = x_val_conti 
    self.all_data = data

  def create_model(self,optimizer = "adam",loss="binary_crossentropy"):
    cat_input = []
    cat_output = []
    for i in range(len(CATEGORICAL_COLUMNS)):
      input = Input(shape= (1,))
      cat_input.append(input)
      dim = len(np.unique(self.all_data[CATEGORICAL_COLUMNS[i]]))
      emb = Embedding(dim, self.embed_dim, input_length =1 ,trainable = True)(input)
      cat_output.append(emb)

    cat_output = Concatenate(axis=1)(cat_output)

    first_order = Flatten()(cat_output)
    first_order = Dense(1, activation='linear')(first_order)

    #cin_output = CIN(cat_output, self.conv_layer)

    dense_input = Input(shape = (len(CONTINUOUS_COLUMNS), ))

    dnn_input = Concatenate(axis=1)([Flatten()(cat_output), dense_input])

    dnn_output = dnn_input 
    for layer in self.hidden_layer:
      dnn_output  = BatchNormalization()(dnn_output)
      dnn_output  = Dense(layer, activation='relu')(dnn_output)
      dnn_output  = Dropout(0.2)(dnn_output)
    dnn_output = Dense(1, activation='linear')(dnn_output)
    output  = Add()([first_order, dnn_output])
    output = Dense(1, activation='sigmoid')(output)
    model = Model(inputs = cat_input + [dense_input], outputs=output)

    model.compile(optimizer=optimizer,loss=loss,metrics=["accuracy"])
    self.model = model
  
  def train(self,batch_size = 64,epochs = 10):
    train_data = [self.x_train_categ[:, i] for i in range(self.x_train_categ.shape[1])] + [self.x_train_conti] 
    val_data = [self.x_val_categ[:, i] for i in range(self.x_train_categ.shape[1])] + [self.x_val_conti]
    self.model.fit(train_data,self.y_train, validation_data=(val_data,self.y_val),epochs = epochs,batch_size=batch_size)
  
  def evaluate(self):
    input_data = [self.x_test_categ[:, i] for i in range(self.x_test_categ.shape[1])] + [self.x_test_conti] 
    loss, acc = self.model.evaluate(input_data, self.y_test)
    print(f'test_loss: {loss} - test_acc: {acc}')


In [0]:
xdeepmf = xDeepMF(hidden_layer=[512,256,128,64],conv_layer=[25, 20, 20],embed_dim=16)

(2357500, 18)


In [0]:
xdeepmf.create_model()

In [0]:
def CIN(x, conv_layer):
    #print('debug :',x.shape)
    cat_output_expand = Lambda(lambda x: K.expand_dims(x, axis = 2))(x)
    #print('debug :',cat_output_expand.shape)
    # shape: -1, 1, feature_size, dim 
    x_0 = Lambda(lambda x: K.permute_dimensions(x, (0,3,2,1)))(cat_output_expand)
    x_next = x_0

    cin_output = []
    for layer in conv_layer:
        z_0 = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=(2,3)))([x_0, x])
        z_1 = Lambda(lambda x: K.permute_dimensions(x, (0,2,3,1)))(z_0)

        x_next_list = []
        pooling_output_list = []
        for index in range(layer):
            output = DepthwiseConv2D((int(z_1.shape[1]), int(z_1.shape[2])))(z_1)
            #output = Conv2D(int(cat_output_expand.shape[-1]), (int(z_1.shape[1]), int(z_1.shape[2])) )(z_1)
            output = Lambda(lambda x: K.squeeze(x, 2))(output)
            pooling_output = Lambda(lambda x: K.sum(output, axis = 2))(output)
            pooling_output_list.append(pooling_output)
            x_next_list.append(output)
        x_next = Concatenate(axis = 1)(x_next_list)
        x_next = Lambda(lambda x: K.expand_dims(x, axis = 2))(x_next)

        x_pooling = Concatenate(axis = 1)(pooling_output_list)
        cin_output.append(x_pooling)

    cin_output =  Concatenate(axis = 1)(cin_output)
    cin_output = Dense(1, activation='linear')(cin_output)
    return cin_output

In [0]:
xdeepmf.train()

Train on 1886000 samples, validate on 235750 samples
Epoch 1/10
 715840/1886000 [==========>...................] - ETA: 3:20 - loss: 0.5375 - accuracy: 0.1751